In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pandas import *
import copy

In [4]:
#Funktioner skrivna av Arend för att skapa en lista med ett tomt bräde och en lista med ett "random" bräde.
def getEmptyBoard():
    return [0,0,0,0,0,0,0,0,0]

def getRandomBoard():
    return list(numpy.random.choice([0,1,2],(9)))

print(getEmptyBoard())

[0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
#Metod för att skriva ut ett bräde med ".", "O" och "X" istället för 0, 1 och 2.
def showBoard(board):
    S=""
    T=[".","O","X"]
    for i in [0,3,6]:
        S=S+T[board[i]]+" "+T[board[i+1]]+" "+T[board[i+2]]
        S=S+"\n"
    print (S)
    
showBoard(getEmptyBoard())
showBoard(getRandomBoard())
print(" 0 1 2")
print(" 3 4 5")
print(" 6 7 8")

. . .
. . .
. . .

O X .
. O X
O O .

 0 1 2
 3 4 5
 6 7 8


In [7]:
#Kollar om någon har vunnit
def score(board):
    line=0
    if board[0]==board[1] and board[1]==board[2] and board[0]!=0:
        line=board[0]
    if board[3]==board[4] and board[4]==board[5] and board[3]!=0:
        line=board[3]
    if board[6]==board[7] and board[7]==board[8] and board[6]!=0:
        line=board[6]
        
    if board[0]==board[3] and board[3]==board[6] and board[0]!=0:
        line=board[0]
    if board[1]==board[4] and board[4]==board[7] and board[1]!=0:
        line=board[1]
    if board[2]==board[5] and board[5]==board[8] and board[2]!=0:
        line=board[2]

    if board[0]==board[4] and board[4]==board[8] and board[0]!=0:
        line=board[0]
    if board[2]==board[4] and board[4]==board[6] and board[2]!=0:
        line=board[2]  
    return line #return 0 for draw or not finished, 1 for O wins, 2 for X wins



#for i in range(3):
#    board=getRandomBoard()
#    showBoard(board)
#    print(score(board))
#    print("---")
board=[1,1,1,0,0,0,2,2,2]
showBoard(board)
print(score(board))

O O O
. . .
X X X

2


In [8]:
#För att göra ett drag skickar man in brädet, vilken ruta draget sker på samt vem som gjorde draget.
def doMove(board,move,who):
    newBoard=copy.deepcopy(board)
    newBoard[move]=who
    return newBoard

board=getEmptyBoard()
showBoard(board)
board=doMove(board,4,1)
showBoard(board)

. . .
. . .
. . .

. . .
. O .
. . .



In [9]:
#Hittar alla möjliga moves genom en loop.
def getAllMoves(board):
    ret=[]
    for i in range(9):
        if board[i]==0:
            ret.append(i)
    return ret

board=getRandomBoard()
showBoard(board)
print(getAllMoves(board))

. . X
O O O
. O .

[0, 1, 6, 8]


In [10]:
#En metod som använder sig av scoremetoden för att kolla om matchen är över, tror inte den används alls.
def gameOver(board):
    s=score(board)
    #Om antalet nollor på brädet är noll eller någon har vunnit enligt funktionen score kommer funktionen returnera true.
    if board.count(0)==0 or s!=0:
        return True
    return False

#Kollar om gameOver fungerar
for i in range(4):
    board=getRandomBoard()
    showBoard(board)
    print(gameOver(board))

O O X
O X O
O O .

True
O O X
. . .
. O .

False
. X X
X O .
. . X

False
X O X
X X X
. X O

True


In [150]:
#Här skapar vi en tom dictionary som ska hålla våran egna policy.
newestAdditionTwo=dict()

#Arends metod för att skapa en randompolicy. Vi kommer använda denna för att skapa ett utgångsläge för våran policy.
def addPossibleBoards(board,who):
    #showBoard(board)
    #Hämtar alla möjliga drag i en lista som ser ut som ex [5, 6, 8]
    moves=getAllMoves(board)
    #V blir en array som innehåller lika många ettor som det finns tillgängliga drag.
    V=numpy.ones(len(moves))
    #Typ som normalisering, skapar sannolikheter
    V=V/V.sum()   
    
    #Kollar så att ingen vunnit och det inte finns noll nollor
    if score(board)==0 and list(board).count(0)!=0:
        #Randomstagegy skapar en tuple av nuvarande brädet och lägger in V.
        newestAdditionTwo[tuple(board)]=V #I dictionaryn lägger vi in nuvarande brädet i en tuple(). Det motsvarar nyckeln i dictionaryn
        #Värdet i dictionaryn blir sannolikheterna. 
        #key(state)=value[sannolikheter]
    flipMove=[0,2,1]
    #En loop som använder en rekursiv funktion.
    #För alla möjliga drag för nuvarande bräde loopas dragen igenom och utförs för nuvarande spelare genom doMovemetoden
    for move in moves:
        newBoard=doMove(board,move,who)
        #När alla drag för spelare who har gjorts kommer nästa runda av drag för nästa spelare att göras.
        addPossibleBoards(newBoard,flipMove[who])
        
#def doMove(board,move,who):
#newBoard=copy.deepcopy(board)
#newBoard[move]=who
#return newBoard
        
#print(newestAddition)
board=getEmptyBoard()
addPossibleBoards(board,1)
print(len(newestAdditionTwo))

4520


In [151]:
import pickle
#pickle.dump(randomStrategy,open("inClassRandomPolicy.p","wb"))
pickle.dump(newestAdditionTwo,open("newestPolicyTwo.p","wb"))
newestPolicyTwo=pickle.load(open("newestPolicy.p","rb"))

print(len(newestPolicy))
randomPolicy=pickle.load(open("inClassRandomPolicy.p","rb"))
perfectPolicy=pickle.load(open("perfectPolicy.p","rb"))

4520


In [128]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [160]:
#Metod för att spela två policier emot varandra, tar två policys som 
def playTwoPolicies(policyA,policyB,verbose=False):
    #Skapar dictionaries för att hålla de olika statesen och sannolikheter som förekommit i matchen som spelats.
    savePlayerOneWinningStrategy = dict() #player O
    savePlayerTwoWinningStrategy = dict() #player X
    savePlayerOneTieStrategy = dict() #player O
    savePlayerTwoTieStrategy = dict() #player X
    
    savedPlayerOneWinningStrategy = dict() #player O
    savedPlayerTwoWinningStrategy = dict() #player X
    savedPlayerOneTieStrategy = dict() #player O
    savedPlayerTwoTieStrategy = dict() #player X
    
    flipMove=[0,2,1]
    who=1
    board=[0,0,0,0,0,0,0,0,0]
    done=False
    wonGameRate = 0.10
    tiedGameRate = 0.01
    lostGameRate = 0
    while not done:
        if verbose:
            showBoard(board)
        moves=getAllMoves(board)
        if who==1:
            p=policyA[tuple(board)]
                        
        else:
            p=policyB[tuple(board)]
        if verbose:
            print(p)
        #Normalisering
        p/=p.sum()
        #Choice 
        choice=numpy.random.choice(moves,p=p) #[0,1,0,0,0,0,0,0,0]
        if who==1:
            #Skapar en lista med nollor som är lika lång som det finns giltiga drag, denna används för att göra drag
            playerOneAction = zeros(len(moves)) #[0,0,0,0,0,0,0,0,0]
            
            #Skapar en lista med nollor som är lika lång som det finns giltiga drag, denna används spara drag
            saveWinningLearningrate = zeros(len(moves)) #[0,0,0,0,0,0,0,0,0]
            saveTyingLearningrate = zeros(len(moves)) #[0,0,0,0,0,0,0,0,0]
            #Hittar platsen i listan man vill lägga in sitt drag i
            sparaIndex = moves.index(choice) #4
            #print(sparaIndex)
            #Lägger in en etta där man vill göra sitt drag.
            playerOneAction[sparaIndex] = 1 #[0,0,0,1,0,0,0,0,0]
                                                
            saveWinningLearningrate[sparaIndex] = wonGameRate #[0,0,0,0.1,0,0,0,0,0]
            saveTyingLearningrate[sparaIndex] = tiedGameRate #[0,0,0,0.01,0,0,0,0,0]
 
            savedWinningAction = copy.deepcopy(saveWinningLearningrate) #[0,0,0,0.1,0,0,0,0,0]  
            savedTyingAction = copy.deepcopy(saveTyingLearningrate) #[0,0,0,0.01,0,0,0,0,0]

            #Sparar action med learningrate till rätt state
            
            savePlayerOneWinningStrategy[tuple(board)]=savedWinningAction #(0, 0, 0, 0, 0, 0, 0, 0, 0): array([0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ])
            savePlayerOneTieStrategy[tuple(board)]=savedTyingAction

        else:
            #Skapar array med nollor
            saveTwoWinningLearningrate = zeros(len(moves))
            saveTwoTyingLearningrate = zeros(len(moves))
            
            #
            playerTwoAction = zeros(len(moves))
            sparaIndex = moves.index(choice)
            playerTwoAction[sparaIndex] = 1
            
            #Uppdaterar learningrates
            saveTwoWinningLearningrate[sparaIndex] = wonGameRate
            saveTwoTyingLearningrate[sparaIndex] = tiedGameRate 
            
            
                        
            savedTwoWinningAction = copy.deepcopy(saveTwoWinningLearningrate)
            savedTwoTyingAction = copy.deepcopy(saveTwoTyingLearningrate)

            
            savePlayerTwoWinningStrategy[tuple(board)]=savedTwoWinningAction
            savePlayerTwoTieStrategy[tuple(board)]=savedTwoTyingAction
        #Här görs draget
        board=doMove(board,choice,who)

            
        s=score(board)
        #Här kollar den om matchen är över?
        if len(moves)==1 or s!=0:
            #showBoard(board)
            if s == 0:
                savePlayerOneTyingStrategy = savePlayerOneTieStrategy
                savePlayerTwoTyingStrategy = savePlayerTwoTieStrategy
                for states in savePlayerOneTyingStrategy.keys():
                    newestPolicyTwo[tuple(states)] += savePlayerOneTyingStrategy[tuple(states)]
                for states in savePlayerTwoTyingStrategy.keys():
                    newestPolicyTwo[tuple(states)] += savePlayerTwoTyingStrategy[tuple(states)]
            if s == 1:
                saveWinningStrategy = savePlayerOneWinningStrategy
                for states in saveWinningStrategy.keys():
                    newestPolicyTwo[tuple(states)] += saveWinningStrategy[tuple(states)]
            if s == 2:
                saveWinningStrategy = savePlayerTwoWinningStrategy
                for states in saveWinningStrategy.keys():
                    newestPolicyTwo[tuple(states)] += saveWinningStrategy[tuple(states)]

            done=True

        #Här flippas turen
        who=flipMove[who]
    if verbose:
        showBoard(board)
    if s==0:
        return 0
    
    return 
for i in range(100000):
    playTwoPolicies(perfectPolicy,randomPolicy,verbose=False)
pickle.dump(newestPolicy,open("newestPolicy.p","wb"))

newestPolicyUpdated=pickle.load(open("newestPolicy.p","rb"))


In [146]:
#print(newestPolicyUpdated)

In [147]:
board=getEmptyBoard()
currentPlayer=1
C=[".","O","X"]
whoWon = 0
#Human vs Human?
savePlayerOneStrategy = dict() #player O
savePlayerTwoStrategy = dict() #player X
saveChoice = []
oldBoard = []
NewBoard = []
playerOneAction = getEmptyBoard()
playerTwoAction = getEmptyBoard()
sparaIndexEtt = []
sparaIndexTva = []

while whoWon == 0:
    showBoard(board)
    print("player "+C[currentPlayer]+" move:")
    if currentPlayer == 1:
        choice=int(input())
        if choice in getAllMoves(board):
            #newBoard = board
            moves=getAllMoves(board)
            print(moves)
            
            playerOneAction = zeros(len(moves))
            sparaIndex = moves.index(choice)
            print(sparaIndex)
            playerOneAction[sparaIndex] = 1
            print(board)
            
            asd = copy.deepcopy(playerOneAction)
            savePlayerOneStrategy[tuple(board)]=asd
            array(asd)
            print(savePlayerOneStrategy)
            board[choice]=currentPlayer
            showBoard(board)
            if score(board) != 0:
                whoWon = currentPlayer
                break
            currentPlayer=2
    if currentPlayer == 2:
        choice=int(input())
        if choice in getAllMoves(board):
            print(choice)
            board[choice]=currentPlayer
            showBoard(board)
            if score(board) != 0:
                whoWon = currentPlayer
                break
            if gameOver(board) == True:
                print('Oavgjort')
                break
            currentPlayer=1
    if score(board) != 0 or len(moves)==0:
        whoWon = currentPlayer
        print(len(moves))
        break
        currentPlayer=3-currentPlayer
showBoard(board)
print(whoWon)
print(savePlayerOneStrategy)
saveWinningStrategy = dict()
if whoWon == 1:
    saveWinningStrategy = savePlayerOneStrategy
if whoWon == 2:
    saveWinningStrategy = savePlayerTwoStrategy

. . .
. . .
. . .

player O move:
0
[0, 1, 2, 3, 4, 5, 6, 7, 8]
0
[0, 0, 0, 0, 0, 0, 0, 0, 0]
{(0, 0, 0, 0, 0, 0, 0, 0, 0): array([1., 0., 0., 0., 0., 0., 0., 0., 0.])}
O . .
. . .
. . .

8
8
O . .
. . .
. . X

O . .
. . .
. . X

player O move:
1
[1, 2, 3, 4, 5, 6, 7]
0
[1, 0, 0, 0, 0, 0, 0, 0, 2]
{(0, 0, 0, 0, 0, 0, 0, 0, 0): array([1., 0., 0., 0., 0., 0., 0., 0., 0.]), (1, 0, 0, 0, 0, 0, 0, 0, 2): array([1., 0., 0., 0., 0., 0., 0.])}
O O .
. . .
. . X

4
4
O O .
. X .
. . X

O O .
. X .
. . X

player O move:
2
[2, 3, 5, 6, 7]
0
[1, 1, 0, 0, 2, 0, 0, 0, 2]
{(0, 0, 0, 0, 0, 0, 0, 0, 0): array([1., 0., 0., 0., 0., 0., 0., 0., 0.]), (1, 0, 0, 0, 0, 0, 0, 0, 2): array([1., 0., 0., 0., 0., 0., 0.]), (1, 1, 0, 0, 2, 0, 0, 0, 2): array([1., 0., 0., 0., 0.])}
O O O
. X .
. . X

O O O
. X .
. . X

1
{(0, 0, 0, 0, 0, 0, 0, 0, 0): array([1., 0., 0., 0., 0., 0., 0., 0., 0.]), (1, 0, 0, 0, 0, 0, 0, 0, 2): array([1., 0., 0., 0., 0., 0., 0.]), (1, 1, 0, 0, 2, 0, 0, 0, 2): array([1., 0., 0., 0., 0.])

In [149]:
def policyVsHuman(policy):
    board=getEmptyBoard()
    currentPlayer=1
    C=[".","O","X"]
    whoWon = 0
    savePlayerOneStrategy = dict() #player O
    savePlayerTwoStrategy = dict() #player X
    saveChoice = []
    oldBoard = []
    NewBoard = []
    playerOneAction = getEmptyBoard()
    playerTwoAction = getEmptyBoard()
    while whoWon == 0:
        print("player "+C[currentPlayer]+" move:")
        if currentPlayer == 1:
            moves=getAllMoves(board)
            print(moves, 'här är alla moves')
            p = policy[tuple(board)]
            p/=p.sum()
            board[numpy.random.choice(moves,p=p)]=currentPlayer#Här är felet
            showBoard(board)
            if score(board) != 0:
                whoWon = currentPlayer
                print(currentPlayer, 'has won the game')
                break
            currentPlayer=2
            print(score(board), '0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit')  
        #insert 1 or 2 into board at location choice
        if board.count(0)==0:
            print('Oavgjort')
            print('whoWon =', whoWon)
            break
        if currentPlayer == 2:        
            choice=int(input())
            if choice in getAllMoves(board):
                newBoard = board
                playerTwoAction[choice] = 2
                asd = copy.deepcopy(playerOneAction)
                savePlayerTwoStrategy[tuple(board)]=asd            
                print(savePlayerTwoStrategy)
                board[choice]=currentPlayer
                if score(board) != 0:
                    whoWon = currentPlayer
                    print(currentPlayer, 'has won the game')
                    break
                currentPlayer=1
                showBoard(board)
                print(score(board), '0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit')
        if board.count(0)==0:
            print('Oavgjort')
            print('whoWon =', whoWon)
            break
    showBoard(board)
    print(whoWon)
    print(savePlayerOneStrategy)
    saveWinningStrategy = dict()
    if whoWon == 1:
        saveWinningStrategy = savePlayerOneStrategy
    if whoWon == 2:
        saveWinningStrategy = savePlayerTwoStrategy
policyVsHuman(newestPolicyUpdated)
#policyVsHuman(perfectPolicy)

player O move:
[0, 1, 2, 3, 4, 5, 6, 7, 8] här är alla moves
. O .
. . .
. . .

0 0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit
2
{(0, 1, 0, 0, 0, 0, 0, 0, 0): [0, 0, 0, 0, 0, 0, 0, 0, 0]}
. O X
. . .
. . .

0 0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit
player O move:
[0, 3, 4, 5, 6, 7, 8] här är alla moves
. O X
. . O
. . .

0 0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit
4
{(0, 1, 0, 0, 0, 0, 0, 0, 0): [0, 0, 0, 0, 0, 0, 0, 0, 0], (0, 1, 2, 0, 0, 1, 0, 0, 0): [0, 0, 0, 0, 0, 0, 0, 0, 0]}
. O X
. X O
. . .

0 0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit
player O move:
[0, 3, 6, 7, 8] här är alla moves
. O X
. X O
O . .

0 0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit
0
{(0, 1, 0, 0, 0, 0, 0, 0, 0): [0, 0, 0, 0, 0, 0, 0, 0, 0], (0, 1, 2, 0, 0, 1, 0, 0, 0): [0, 0, 0, 0, 0, 0, 0, 0, 0], (0, 1, 2, 0, 2, 1, 1, 0, 0): [0, 0, 0, 0, 0, 0, 0, 0, 0]}
X O X
. X O
O . .

0 0 = ingen vinnare, 1-2=spelare 1 eller 2 har vunnit
player O move:
[3, 7, 8] här är alla mov

In [76]:
def originalPlayTwoPolicies(policyA,policyB,verbose=False):
    flipMove=[0,2,1]
    who=1
    board=[0,0,0,0,0,0,0,0,0]
    done=False
    while not done:
        if verbose:
            showBoard(board)
        moves=getAllMoves(board)
        if who==1:
            p=policyA[tuple(board)]
        else:
            p=policyB[tuple(board)]
        if verbose:
            print(p)
        p/=p.sum()
        choice=numpy.random.choice(moves,p=p)
        board=doMove(board,choice,who)
        s=score(board)
        if len(moves)==1 or s!=0:
            done=True
        who=flipMove[who]
    if verbose:
        showBoard(board)
    if s==0:
        return 0
    return flipMove[who]

In [163]:
print(originalPlayTwoPolicies(testatForMycket,randomPolicy,verbose=False))

NameError: name 'testatForMycket' is not defined

In [161]:
def sampleGames(policyA,policyB,nrOfGames=100):
    result=[0,0,0]
    for n in range(nrOfGames):
        winner=originalPlayTwoPolicies(policyA,policyB)
        result[winner]+=1
    result=numpy.array(result)
    return result/result.sum()

#print("RvR",sampleGames(newestPolicyUpdated,perfectPolicy,nrOfGames=1000))
#print("PvP",sampleGames(perfectPolicy,perfectPolicy,nrOfGames=100000))
#print("PvR",sampleGames(perfectPolicy,randomPolicy,nrOfGames=100))
print("RvP",sampleGames(randomPolicy,randomPolicy,nrOfGames=100))

RvP [0.13 0.59 0.28]


In [162]:
#RvR [0.996 0.    0.004]
#RvR [0.035 0.964 0.001]
#RvR [0.047 0.951 0.002]
#RvR [0.054 0.94  0.006]
#RvR [0.032 0.964 0.004]
#RvR [0.062 0.895 0.043]
#RvR [0.096 0.772 0.132]
#RvR [0.077 0.789 0.134]
#RvR [0.072 0.812 0.116]
#testPolicy=pickle.load(open("forsoksPolicy.p","rb"))
#print(forsoksPolicy)
print(newestPolicyUpdated)

{(0, 0, 0, 0, 0, 0, 0, 0, 0): array([1.11111111e-01, 3.86183811e+04, 1.11111111e-01, 1.11111111e-01,
       1.11111111e-01, 1.11111111e-01, 1.11111111e-01, 1.11111111e-01,
       1.11111111e-01]), (1, 0, 0, 0, 0, 0, 0, 0, 0): array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]), (1, 2, 0, 0, 0, 0, 0, 0, 0): array([0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
       0.14285714, 0.14285714]), (1, 2, 1, 0, 0, 0, 0, 0, 0): array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667]), (1, 2, 1, 2, 0, 0, 0, 0, 0): array([0.2, 0.2, 0.2, 0.2, 0.2]), (1, 2, 1, 2, 1, 0, 0, 0, 0): array([0.25, 0.25, 0.25, 0.25]), (1, 2, 1, 2, 1, 2, 0, 0, 0): array([0.33333333, 0.33333333, 0.33333333]), (1, 2, 1, 2, 1, 2, 0, 1, 0): array([0.5, 0.5]), (1, 2, 1, 2, 1, 2, 2, 1, 0): array([1.]), (1, 2, 1, 2, 1, 2, 0, 1, 2): array([1.]), (1, 2, 1, 2, 1, 0, 2, 0, 0): array([0.33333333, 0.33333333, 0.33333333]), (1, 2, 1, 2, 1, 1, 2, 0, 0): array([0.5, 0.5]), (1, 2, 1, 